## 1. **BFMatcher (Brute-Force Matcher)**
Compares descriptors between keypoints in a brute-force manner, for distance calculation, such as `cv::NORM_L2` or `cv::NORM_HAMMING`. It is suitable for small datasets.

**Distance Types**
The "distance" is purely a measure of similarity between feature descriptors, (Smaller distance = Higher similarity).
It does not directly correspond to spatial distances in pixels.

1. **`cv::NORM_L2`, `cv.NORM_L2` (Euclidean Distance):**   Euclidean distance, used mainly for floating-point descriptors like **SIFT/SURF**. By default, it is `cv.NORM_L2`. It is good for SIFT, SURF etc (`cv.NORM_L1 is` also there).  Distance = √(Σ(descriptor1[i] - descriptor2[i])²)

```
bf = cv2.BFMatcher(cv2.cv.NORM_L2, crossCheck=True)
```

2. **`cv::NORM_HAMMING,cv.NORM_HAMMING` (Hamming Distance):** Used for binary string-based descriptors like **ORB/BRIEF/BRISK**. It counts the number of differing bits between two binary strings. If ORB is using `WTA_K == 3` or `4`, which takes 3 or 4 points to produce BRIEF descriptor


```python
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
```

`Cross Check`: An option in the BFMatcher that ensures mutual matching. For two keypoints to be considered a match, the keypoint in the first image must match the keypoint in the second image, and vice-versa.



#### 1.2 Simple `BFMatcher`
```cpp
cv::Ptr<cv::Feature2D> detector = cv::ORB::create();

cv::BFMatcher bfMatcher(cv::NORM_HAMMING);
std::vector<cv::DMatch> bfMatches;
bfMatcher.match(descriptors1, descriptors2, bfMatches);
```


<img src="images/BFMatcher.png" />

#### 1.3 `knnMatcher`

```cpp
// Use BFMatcher with NORM_HAMMING (suitable for ORB descriptors)
cv::BFMatcher matcher(cv::NORM_HAMMING, /*crossCheck=*/false);

// Perform KNN matching (k=2)
std::vector<std::vector<cv::DMatch>> knnMatches;
matcher.knnMatch(descriptors1, descriptors2, knnMatches, k);

// Apply Lowe's ratio test to filter matches
const float ratioThresh = 0.75f; // Lowe's ratio test threshold
std::vector<cv::DMatch> goodMatches;
for (const auto& knnMatch : knnMatches) {
    if (knnMatch.size() >= 2 && knnMatch[0].distance < ratioThresh * knnMatch[1].distance) {
        goodMatches.push_back(knnMatch[0]);
    }
}
```


<img src="images/KNN_Matches.png" />




#### 1.3 `radiusMatch`
Use `radiusMatch` when you want to consider all matches within a **spatial or distance threshold**, especially for applications where proximity is more critical than ranking.


If you’re using ORB and set `maxDistance = 50` in `radiusMatch`:
- The matcher will consider all descriptors from the second image that are within 50 bits difference (Hamming distance) of a descriptor from the first image.

If you’re using SIFT and set `maxDistance = 1.0`:
- The matcher will consider all descriptors within a Euclidean distance of 1.0.


```cpp
cv::BFMatcher matcher(cv::NORM_HAMMING);

// Perform radius matching
const float maxDistance = 50.0f; // Radius threshold
std::vector<std::vector<cv::DMatch>> radiusMatches;
matcher.radiusMatch(descriptors1, descriptors2, radiusMatches, maxDistance);

// Filter and collect matches for visualization
std::vector<cv::DMatch> goodMatches;
for (const auto& matches : radiusMatches) {
    for (const auto& match : matches) {
        if (match.distance < maxDistance) {
            goodMatches.push_back(match);
        }
    }
}
cv::Mat imgMatches;
cv::drawMatches(img1, keypoints1, img2, keypoints2, goodMatches, imgMatches);
```

Here, a `std::vector<std::vector<cv::DMatch>>` is used because this method can potentially return multiple matches for each query descriptor. Unlike other matching methods that return a single best match,  `radiusMatch` finds all matches within a specified distance (`maxDistance`) for each query descriptor  in `descriptors2` (train set) that are within the given radius.

- The outer vector corresponds to each descriptor in the query set (`descriptors1`).
- The inner vector contains all matches found within the specified radius for that query descriptor.





<img src="images/Radius_Match.png" />

## 2. **FlannBasedMatcher (Fast Library for Approximate Nearest Neighbors)**
Efficient for large datasets by using an approximate nearest neighbor algorithm, suitable for high-dimensional data and faster than BFMatcher for larger datasets.



```cpp
cv::FlannBasedMatcher flannMatcher;
std::vector<cv::DMatch> flannMatches;
flannMatcher.match(descriptors1, descriptors2, flannMatches);
```

### 3. `cv::DMatch` 
is a data structure in OpenCV that represents a match between two keypoints from different images, typically used in feature matching processes. It contains information about the correspondence between these keypoints, such as their indices and the quality of the match.

**Members of `cv::DMatch`**
1. **`int queryIdx`**  
   - The index of the keypoint in the query image (the image from which you are searching for matches).
   - It refers to a keypoint in the keypoint vector provided for the query image.

2. **`int trainIdx`**  
   - The index of the keypoint in the train image (the image being searched for matches).
   - It refers to a keypoint in the keypoint vector provided for the train image.

3. **`int imgIdx`**  
   - The index of the image in the training dataset if you are using a collection of images. 
   - In most cases, this is not used directly when matching a single image to another, as `trainIdx` suffices.

4. **`float distance`**  
   - A measure of how well the descriptors of the two keypoints match.
   - Typically, this is the Euclidean distance between the descriptor vectors of the two keypoints. A smaller distance indicates a better match.

- `queryIdx` and `trainIdx` link the match to keypoints in the respective images.
- `distance` is the main criterion for assessing the quality of a match. Matches with smaller distances are generally better.
- Matches can be filtered based on the `distance` or other criteria (e.g., using Lowe's ratio test for better robustness).

## 4. Correspondence match refinement and rejection

### 4.1 Lowe's Ratio Test

It is a **validation step** that helps distinguish between good matches (true correspondences) and poor or ambiguous matches (false correspondences).The idea is based on the assumption that a good match should have a **significantly better similarity score** (smaller distance) than the next best alternative match.



#### How Does it Work?

1. **`knnMatch` Results**:
   - For each descriptor in the first image, the `knnMatch` function retrieves the top `k` closest matches from the descriptors in the second image.
   - Typically, `k=2` is used to retrieve the two best matches: `knnMatch[0]` (best match) and `knnMatch[1]` (second-best match).

2. **Ratio Test**:
   - Compare the distance of the best match (`knnMatch[0].distance`) to the distance of the second-best match (`knnMatch[1].distance`).
   - If the ratio of these distances is below a predefined threshold (e.g., 0.75), it indicates a strong match. Otherwise, the match is considered ambiguous and discarded.

   Mathematically:
   $
   \text{If } \frac{\text{knnMatch[0].distance}}{\text{knnMatch[1].distance}} < \text{ratioThresh}, \text{ accept the match.}
   $

   - **Typical `ratioThresh` Values**:
     - The default threshold is often **0.7** or **0.75** (Lowe's original paper recommends 0.75).



```cpp
const float ratioThresh = 0.75f; // Lowe's ratio threshold
std::vector<cv::DMatch> goodMatches;

for (const auto& knnMatch : knnMatches) {
    if (knnMatch.size() >= 2) { // Ensure at least two matches exist
        // Apply Lowe's ratio test
        if (knnMatch[0].distance < ratioThresh * knnMatch[1].distance) {
            goodMatches.push_back(knnMatch[0]);
        }
    }
}
```

---



#### Example:
Imagine the distances for a descriptor in image 1 are:
- `knnMatch[0].distance = 0.3` (best match)
- `knnMatch[1].distance = 0.5` (second-best match)

$
\text{Ratio: } \frac{0.3}{0.5} = 0.6
$

If `ratioThresh = 0.75`, this match passes the test.

#### Counterexample:
Now imagine the distances are:
- `knnMatch[0].distance = 0.4`
- `knnMatch[1].distance = 0.42`

$
\text{Ratio: } \frac{0.4}{0.42} \approx 0.95
$

This match fails the test because the best match is not significantly better than the second-best match, indicating ambiguity.


```python
matches = matcher.knnMatch(des1, des2, k=2)
good_matches = []

# Ratio Test:  For each keypoint, if the distance ratio between the best and the second-best match is below a
# threshold (usually around 0.7 to 0.8), the match is retained.
ratio_rest = 0.7
for m, n in matches:
    if m.distance < ratio_rest * n.distance:
        good_matches.append(m)
matches = good_matches
```

### 4.2 Quantile-based filtering

- If you choose "2", you are asking for all elements below the median (50th percentile).
- If you choose "4", you are asking for elements below the 25th percentile (1/4 of the sorted data).


```cpp
    std::vector<cv::DMatch> matches;
    matcher.match(descriptors[i], descriptors[i + 1], matches);
    std::vector<float> distances;
    for (const auto &match : matches) {
      distances.push_back(match.distance);
    }
    std::nth_element(distances.begin(),
                     distances.begin() + distances.size() / quantiles,
                     distances.end());
    float median_distance = distances[distances.size() / quantiles];

    matches.erase(std::remove_if(matches.begin(), matches.end(),
                                 [median_distance](const cv::DMatch &match) {
                                   return match.distance > median_distance;
                                 }),
                  matches.end());
```

```python
kp1, des1 = detector.detectAndCompute(img1, None)
kp2, des2 = detector.detectAndCompute(img2, None)

matches = matcher.match(des1, des2)

# Sort them based on the distance
matches = sorted(matches, key=lambda x: x.distance)

matches = matches[:int(len(matches)/top_k_matches)]
```

## 5. Drawing Matches

```cpp
cv::drawMatches(img1, keypoints1, img2, keypoints2, matches, img_matches);
```


```python
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2)
```

[c++ code](../src/correspondences_matching.cpp)

[python code](../scripts/correspondences_matching.py)